# Getting results from ElasticSearch
# Data cleaning and concatenating using pandas

Recommend installing the following libraries:
- Pandas, Numpy, matplotlib (from Anaconda3)
- elasticsearch

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from elasticsearch import Elasticsearch

In [2]:
es = Elasticsearch(['http://eep16.fcr-it.top:9200'])
es_index="fscrawler-mount"

In [3]:
search_term = "hsbc exchange rate forecast g10"

In [17]:
# highlighting doesn't work quite well yet.
# keep on studying...

res = es.search(
        index=es_index,
        size=20,
        body={
             "query": {
                "simple_query_string" : {
                    "query": search_term,
                    "fields": ["content"],
                    "default_operator": "and"
                }
              }       
        }
    )

In [18]:
# Show the ElasticSearch result format
res['hits']['hits']

[{'_id': 'c5d06796845f6f6a20eb495345ce4a46',
  '_index': 'fscrawler-mount',
  '_score': 5.4263706,
  '_source': {'content': '\n \n\nDisclaimer & Disclosures \n\nThis report must be read with the disclosures and the analyst certifications in \n\nthe Disclosure appendix, and with the Disclaimer, which forms part of it. \n\n  \n\n \n\nIssuer of report: HSBC Bank plc \n\nView HSBC Global Research at: \n\nhttps://www.research.hsbc.com \n\n\uf061\uf062\uf063\uf020\n\uf020\n\nTHIS CONTENT MAY NOT BE DISTRIBUTED TO THE PEOPLE\'S REPUBLIC OF CHINA (THE "PRC") \n\n(EXCLUDING SPECIAL ADMINISTRATIVE REGIONS OF HONG KONG AND MACAO) \n\n \n\n\uf034 After the recent equity sell-off, our analysis suggests that a \n\nnew volatility wave is under way \n\n\uf034 We discuss cross-asset implications of rising volatility…  \n\n\uf034 …and also address the apparent RORO resurgence \n\nPart I: The return of volatility \n\nThe recent market sell-off suggests that volatility is back. After the pick-up in EM \n\

In [23]:
# Show in pandas format
df = pd.DataFrame(res['hits']['hits'])
# Split the _source column
df = pd.concat([df.drop(['_source'], axis=1), df['_source'].apply(pd.Series)], axis=1)
df = pd.concat([df.drop(['meta'], axis=1), df['meta'].apply(pd.Series)], axis=1)
#df = pd.concat([df.drop(['path'], axis=1), df['path'].apply(pd.Series)], axis=1)
#df = pd.concat([df.drop(['file'], axis=1), df['file'].apply(pd.Series)], axis=1)
#df = pd.concat([df.drop(['raw'], axis=1), df['raw'].apply(pd.Series)], axis=1)
#df = pd.concat([df,df.content.str.slice(600,900).apply(pd.Series)], axis=1)
df = pd.concat([df,df["content"].str.slice(600,900).rename("my_value")], axis=1)
df

#df2 = df["content"].str.slice(600,900).rename("my_value")
#df2

,_id,_index,_score,_type,content,file,path,date,language,format,creator_tool,created,raw,my_value
0,c5d06796845f6f6a20eb495345ce4a46,fscrawler-mount,5.426371,_doc,\n \n\nDisclaimer & Disclosures \n\nThis repor...,"{'extension': 'pdf', 'content_type': 'applicat...","{'root': '7c31416ded04ecb6137ddc062de7dcb', 'v...",2018-10-24T07:36:34.000+0000,en-US,application/pdf; version=1.6,Microsoft® Word 2013,2018-10-24T03:03:26.000+0000,"{'date': '2018-10-24T07:36:34Z', 'pdf:PDFVersi...",rising volatility… \n\n …and also address th...


In [21]:
# Try to check if the content column contains specific string.
# May further enhance it to match the string, and get the subsequent 100 characters and show that as a result
# This can reduce the content in each row to be shown in the UI

# Reference：  http://pandas.pydata.org/pandas-docs/stable/text.html#extracting-substrings

pattern = 'Disclaimer'
#df.content.str.contains(pattern)
df.content.str.slice(600,900)


0    rising volatility…  \n\n …and also address th...
Name: content, dtype: object

In [22]:
df.T.to_dict()

{0: {'_id': 'c5d06796845f6f6a20eb495345ce4a46',
  '_index': 'fscrawler-mount',
  '_score': 5.4263706,
  '_type': '_doc',
  'content': '\n \n\nDisclaimer & Disclosures \n\nThis report must be read with the disclosures and the analyst certifications in \n\nthe Disclosure appendix, and with the Disclaimer, which forms part of it. \n\n  \n\n \n\nIssuer of report: HSBC Bank plc \n\nView HSBC Global Research at: \n\nhttps://www.research.hsbc.com \n\n\uf061\uf062\uf063\uf020\n\uf020\n\nTHIS CONTENT MAY NOT BE DISTRIBUTED TO THE PEOPLE\'S REPUBLIC OF CHINA (THE "PRC") \n\n(EXCLUDING SPECIAL ADMINISTRATIVE REGIONS OF HONG KONG AND MACAO) \n\n \n\n\uf034 After the recent equity sell-off, our analysis suggests that a \n\nnew volatility wave is under way \n\n\uf034 We discuss cross-asset implications of rising volatility…  \n\n\uf034 …and also address the apparent RORO resurgence \n\nPart I: The return of volatility \n\nThe recent market sell-off suggests that volatility is back. After the pick-up